In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import re
import collections
import pickle
import random
import os
import time
import argparse
import math
import copy

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn

In [5]:
from gensim.models.word2vec import Word2Vec

## 1. Generator

In [6]:
class Generator(nn.Module):
    """ Generator """

    def __init__(self, vocab_size, embedding_dim, hidden_dim, use_cuda):
        super(Generator, self).__init__()
        self.hidden_dim = hidden_dim
        self.use_cuda = use_cuda
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
        self.log_softmax = nn.LogSoftmax(dim=1)
        self.init_params()

    def forward(self, x):
        """
        Embeds input and applies LSTM on the input sequence.
        Inputs: x
            - x: (batch_size, seq_len), sequence of tokens generated by generator
        Outputs: out
            - out: (batch_size * seq_len, vocab_size), lstm output prediction
        """
        self.lstm.flatten_parameters()
        h0, c0 = self.init_hidden(x.size(0))
        emb = self.embed(x) # batch_size * seq_len * emb_dim 
        out, _ = self.lstm(emb, (h0, c0)) # out: batch_size * seq_len * hidden_dim
        out = self.log_softmax(self.fc(out.contiguous().view(-1, self.hidden_dim))) # (batch_size*seq_len) * vocab_size
        return out

    def step(self, x, h, c):
        """
        Embeds input and applies LSTM one token at a time (seq_len = 1).
        Inputs: x, h, c
            - x: (batch_size, 1), sequence of tokens generated by generator
            - h: (1, batch_size, hidden_dim), lstm hidden state
            - c: (1, batch_size, hidden_dim), lstm cell state
        Outputs: out, h, c
            - out: (batch_size, vocab_size), lstm output prediction
            - h: (1, batch_size, hidden_dim), lstm hidden state
            - c: (1, batch_size, hidden_dim), lstm cell state 
        """
        self.lstm.flatten_parameters()
        emb = self.embed(x) # batch_size * 1 * emb_dim
        out, (h, c) = self.lstm(emb, (h, c)) # out: batch_size * 1 * hidden_dim
        out = self.log_softmax(self.fc(out.contiguous().view(-1, self.hidden_dim))) # batch_size * vocab_size
        return out, h, c

    def init_hidden(self, batch_size):
        h = torch.zeros(1, batch_size, self.hidden_dim)
        c = torch.zeros(1, batch_size, self.hidden_dim)
        if self.use_cuda:
            h, c = h.cuda(), c.cuda()
        return h, c
    
    def init_params(self):
        for param in self.parameters():
            param.data.uniform_(-0.05, 0.05)

    def sample(self, batch_size, seq_len, x=None):
        """
        Samples the network and returns a batch of samples of length seq_len.
        Outputs: out
            - out: (batch_size * seq_len)
        """
        samples = []
        if x is None:
            h, c = self.init_hidden(batch_size)
            x = torch.zeros(batch_size, 1, dtype=torch.int64)
            if self.use_cuda:
                x = x.cuda()
            for _ in range(seq_len):
                out, h, c = self.step(x, h, c)
                prob = torch.exp(out)
                x = torch.multinomial(prob, 1)
                samples.append(x)
        else:
            h, c = self.init_hidden(x.size(0))
            given_len = x.size(1)
            lis = x.chunk(x.size(1), dim=1)
            for i in range(given_len):
                out, h, c = self.step(lis[i], h, c)
                samples.append(lis[i])
            prob = torch.exp(out)
            x = torch.multinomial(prob, 1)
            for _ in range(given_len, seq_len):
                samples.append(x)
                out, h, c = self.step(x, h, c)
                prob = torch.exp(out)
                x = torch.multinomial(prob, 1)
        out = torch.cat(samples, dim=1) # along the batch_size dimension
        return out

## 2. Discriminator

In [7]:
class Discriminator(nn.Module):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    Highway architecture based on the pooled feature maps is added. Dropout is adopted.
    """

    def __init__(self, num_classes, vocab_size, embedding_dim, filter_sizes, num_filters, dropout_prob):
        super(Discriminator, self).__init__()
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([
            nn.Conv2d(1, num_f, (f_size, embedding_dim)) for f_size, num_f in zip(filter_sizes, num_filters)
        ])
        self.highway = nn.Linear(sum(num_filters), sum(num_filters))
        self.dropout = nn.Dropout(p = dropout_prob)
        self.fc = nn.Linear(sum(num_filters), num_classes)

    def forward(self, x):
        """
        Inputs: x
            - x: (batch_size, seq_len)
        Outputs: out
            - out: (batch_size, num_classes)
        """
        emb = self.embed(x).unsqueeze(1) # batch_size, 1 * seq_len * emb_dim
        convs = [F.relu(conv(emb)).squeeze(3) for conv in self.convs] # [batch_size * num_filter * seq_len]
        pools = [F.max_pool1d(conv, conv.size(2)).squeeze(2) for conv in convs] # [batch_size * num_filter]
        out = torch.cat(pools, 1)  # batch_size * sum(num_filters)
        highway = self.highway(out)
        transform = F.sigmoid(highway)
        out = transform * F.relu(highway) + (1. - transform) * out # sets C = 1 - T
        out = F.log_softmax(self.fc(self.dropout(out)), dim=1) # batch * num_classes
        return out

## 3. Target LSTM

In [8]:
class TargetLSTM(nn.Module):
    """ Target LSTM """

    def __init__(self,  vocab_size, embedding_dim, hidden_dim, use_cuda):
        super(TargetLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.use_cuda = use_cuda
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
        self.log_softmax = nn.LogSoftmax(dim=1)
        self.init_params()

    def forward(self, x):
        """
        Embeds input and applies LSTM on the input sequence.
        Inputs: x
            - x: (batch_size, seq_len), sequence of tokens generated by generator
        Outputs: out
            - out: (batch_size, vocab_size), lstm output prediction
        """
        self.lstm.flatten_parameters()
        h0, c0 = self.init_hidden(x.size(0))
        emb = self.embed(x) # batch_size * seq_len * emb_dim 
        out, _ = self.lstm(emb, (h0, c0)) # out: seq_len * batch_size * hidden_dim
        out = self.log_softmax(self.fc(out.contiguous().view(-1, self.hidden_dim))) # seq_len * batch_size * vocab_size
        return out

    def step(self, x, h, c):
        """
        Embeds input and applies LSTM one token at a time (seq_len = 1).
        Inputs: x, h, c
            - x: (batch_size, 1), sequence of tokens generated by generator
            - h: (1, batch_size, hidden_dim), lstm hidden state
            - c: (1, batch_size, hidden_dim), lstm cell state
        Outputs: out, h, c
            - out: (batch_size, 1, vocab_size), lstm output prediction
            - h: (1, batch_size, hidden_dim), lstm hidden state
            - c: (1, batch_size, hidden_dim), lstm cell state 
        """
        self.lstm.flatten_parameters()
        emb = self.embed(x) # batch_size * 1 * emb_dim
        out, (h, c) = self.lstm(emb, (h, c)) # out: batch_size * 1 * hidden_dim
        out = self.log_softmax(self.fc(out.contiguous().view(-1, self.hidden_dim))) # batch_size * vocab_size
        return out, h, c

    def init_hidden(self, batch_size):
        h = torch.zeros((1, batch_size, self.hidden_dim))
        c = torch.zeros((1, batch_size, self.hidden_dim))
        if self.use_cuda:
            h, c = h.cuda(), c.cuda()
        return h, c
    
    def init_params(self):
        for param in self.parameters():
            param.data.normal_(0, 1)

    def sample(self, batch_size, seq_len):
        """
        Samples the network and returns a batch of samples of length seq_len.
        Outputs: out
            - out: (batch_size * seq_len)
        """
        samples = []
        h, c = self.init_hidden(batch_size)
        x = torch.zeros(batch_size, 1, dtype=torch.int64)
        if self.use_cuda:
            x = x.cuda()
        for _ in range(seq_len):
            out, h, c = self.step(x, h, c)
            prob = torch.exp(out)
            x = torch.multinomial(prob, 1)
            samples.append(x)
        out = torch.cat(samples, dim=1) # along the batch_size dimension
        return out

## 4. PGLoss

In [8]:
class PGLoss(nn.Module):
    """
    Pseudo-loss that gives corresponding policy gradients (on calling .backward()) 
    for adversial training of Generator
    """

    def __init__(self):
        super(PGLoss, self).__init__()

    def forward(self, pred, target, reward):
        """
        Inputs: pred, target, reward
            - pred: (batch_size, seq_len), 
            - target : (batch_size, seq_len), 
            - reward : (batch_size, ), reward of each whole sentence
        """
        one_hot = torch.zeros(pred.size(), dtype=torch.uint8)
        if pred.is_cuda:
            one_hot = one_hot.cuda()
        one_hot.scatter_(1, target.data.view(-1, 1), 1)
        loss = torch.masked_select(pred, one_hot)
        loss = loss * reward.contiguous().view(-1)
        loss = -torch.sum(loss)
        return loss

##5. Data Iter

In [9]:
class GenDataIter:
    """ Toy data iter to load digits """

    def __init__(self, data_file, batch_size):
        super(GenDataIter, self).__init__()
        self.batch_size = batch_size
        self.data_lis = self.read_file(data_file)
        self.data_num = len(self.data_lis)
        self.indices = range(self.data_num)
        self.num_batches = math.ceil(self.data_num / self.batch_size)
        self.idx = 0
        self.reset()

    def __len__(self):
        return self.num_batches

    def __iter__(self):
        return self

    def __next__(self):
        return self.next()
    
    def reset(self):
        self.idx = 0
        random.shuffle(self.data_lis)

    def next(self):
        if self.idx >= self.data_num:
            raise StopIteration
        index = self.indices[self.idx : self.idx + self.batch_size]
        d = [self.data_lis[i] for i in index]
        d = torch.tensor(d)

        # 0 is prepended to d as start symbol
        data = torch.cat([torch.zeros(len(index), 1, dtype=torch.int64), d], dim=1)
        target = torch.cat([d, torch.zeros(len(index), 1, dtype=torch.int64)], dim=1)
        
        self.idx += self.batch_size
        return data, target

    def read_file(self, data_file):
        with open(data_file, 'r') as f:
            lines = f.readlines()
        lis = []
        for line in lines:
            l = [int(s) for s in list(line.strip().split())]
            lis.append(l)
        return lis


class DisDataIter:
    """ Toy data iter to load digits """

    def __init__(self, real_data_file, fake_data_file, batch_size):
        super(DisDataIter, self).__init__()
        self.batch_size = batch_size
        real_data_lis = self.read_file(real_data_file)
        fake_data_lis = self.read_file(fake_data_file)
        self.data = real_data_lis + fake_data_lis
        self.labels = [1 for _ in range(len(real_data_lis))] +\
                        [0 for _ in range(len(fake_data_lis))]
        self.pairs = list(zip(self.data, self.labels))
        self.data_num = len(self.pairs)
        self.indices = range(self.data_num)
        self.num_batches = math.ceil(self.data_num / self.batch_size)
        self.idx = 0
        self.reset()

    def __len__(self):
        return self.num_batches

    def __iter__(self):
        return self

    def __next__(self):
        return self.next()
    
    def reset(self):
        self.idx = 0
        random.shuffle(self.pairs)

    def next(self):
        if self.idx >= self.data_num:
            raise StopIteration
        index = self.indices[self.idx : self.idx + self.batch_size]
        pairs = [self.pairs[i] for i in index]
        data = [p[0] for p in pairs]
        label = [p[1] for p in pairs]
        data = torch.tensor(data)
        label = torch.tensor(label)
        self.idx += self.batch_size
        return data, label

    def read_file(self, data_file):
        with open(data_file, 'r') as f:
            lines = f.readlines()
        lis = []
        for line in lines:
            l = [int(s) for s in list(line.strip().split())]
            lis.append(l) 
        return lis

## 5. Rollout

In [10]:
class Rollout(object):
    """ Rollout Policy """

    def __init__(self, model, update_rate):
        self.ori_model = model
        self.own_model = copy.deepcopy(model)
        self.update_rate = update_rate

    def get_reward(self, x, num, discriminator):
        """
        Inputs: x, num, discriminator
            - x: (batch_size, seq_len) input data
            - num: rollout number
            - discriminator: discrimanator model
        """
        rewards = []
        batch_size = x.size(0)
        seq_len = x.size(1)
        for i in range(num):
            for l in range(1, seq_len):
                data = x[:, 0:l]
                samples = self.own_model.sample(batch_size, seq_len, data)
                pred = discriminator(samples)
                pred = pred.cpu().data[:,1].numpy()
                if i == 0:
                    rewards.append(pred)
                else:
                    rewards[l-1] += pred

            # for the last token
            pred = discriminator(x)
            pred = pred.cpu().data[:, 1].numpy()
            if i == 0:
                rewards.append(pred)
            else:
                rewards[seq_len-1] += pred
        rewards = np.transpose(np.array(rewards)) / (1.0 * num) # batch_size * seq_len
        return rewards

    def update_params(self):
        dic = {}
        for name, param in self.ori_model.named_parameters():
            dic[name] = param.data
        for name, param in self.own_model.named_parameters():
            if name.startswith('emb'):
                param.data = dic[name]
            else:
                param.data = self.update_rate * param.data + (1 - self.update_rate) * dic[name]

## 6. seqGAN

In [16]:
#########################################################################################
#  Generator  Hyper-parameters
######################################################################################
g_embed_dim = 200 # embedding dimension (pretrained: 200, pk: 30)
g_hidden_dim = 300 # hidden state dimension of lstm cell
g_seq_len = 20 # sequence length
START_TOKEN = 0

#########################################################################################
#  Discriminator  Hyper-parameters
#########################################################################################
d_num_class = 2
d_embed_dim = 64
d_filter_sizes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20]
d_num_filters = [100, 200, 200, 200, 200, 100, 100, 100, 100, 100, 160, 160]
d_dropout_prob = 0.75


POSITIVE_FILE = 'real_data.txt'
NEGATIVE_FILE = 'gene_data.txt'

def generate_samples(model, batch_size, generated_num, output_file):
    samples = []
    for _ in range(int(generated_num / batch_size)):
        sample = model.sample(batch_size, g_seq_len).cpu().data.numpy().tolist()
        samples.extend(sample)
    with open(output_file, 'w') as fout:
        for sample in samples:
            string = ' '.join([str(s) for s in sample])
     
            fout.write('{}\n'.format(string))


def train_generator_MLE(gen, data_iter, criterion, optimizer, epochs, 
        gen_pretrain_train_loss, args):
    """
    Train generator with MLE
    """
    for epoch in range(epochs):
        total_loss = 0.
        for data, target in data_iter:
            if args.cuda:
                data, target = data.cuda(), target.cuda()
            target = target.contiguous().view(-1)
            output = gen(data)
            loss = criterion(output, target)
            total_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        data_iter.reset()
    avg_loss = total_loss / len(data_iter)
    print("Epoch {}, train loss: {:.5f}".format(epoch, avg_loss))
    gen_pretrain_train_loss.append(avg_loss)

def train_generator_PG(gen, dis, rollout, pg_loss, optimizer, epochs, args):
    """
    Train generator with the guidance of policy gradient
    """
    for epoch in range(epochs):
        # construct the input to the genrator, add zeros before samples and delete the last column
        samples = generator.sample(args.batch_size, g_seq_len)
        zeros = torch.zeros(args.batch_size, 1, dtype=torch.int64)
        if samples.is_cuda:
            zeros = zeros.cuda()
        inputs = torch.cat([zeros, samples.data], dim = 1)[:, :-1].contiguous()
        targets = samples.data.contiguous().view((-1,))

        # calculate the reward
        rewards = torch.tensor(rollout.get_reward(samples, args.n_rollout, dis))
        if args.cuda:
            rewards = rewards.cuda()

        # update generator
        output = gen(inputs)
        loss = pg_loss(output, targets, rewards)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def eval_generator(model, data_iter, criterion, args):
    """
    Evaluate generator with NLL
    """
    total_loss = 0.
    with torch.no_grad():
        for data, target in data_iter:
            if args.cuda:
                data, target = data.cuda(), target.cuda()
            target = target.contiguous().view(-1)
            pred = model(data)
            loss = criterion(pred, target)
            total_loss += loss.item()
    avg_loss = total_loss / len(data_iter)
    return avg_loss


def train_discriminator(dis, gen, criterion, optimizer, epochs, 
        dis_adversarial_train_loss, dis_adversarial_train_acc, args):
    """
    Train discriminator
    """
    generate_samples(gen, args.batch_size, args.n_samples, NEGATIVE_FILE)
    data_iter = DisDataIter(POSITIVE_FILE, NEGATIVE_FILE, args.batch_size)
    for epoch in range(epochs):
        correct = 0
        total_loss = 0.
        for data, target in data_iter:
            if args.cuda:
                data, target = data.cuda(), target.cuda()
            target = target.contiguous().view(-1)
            output = dis(data)
            pred = output.data.max(1)[1]
            correct += pred.eq(target.data).cpu().sum()
            loss = criterion(output, target)
            total_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        data_iter.reset()
        avg_loss = total_loss / len(data_iter)
        acc = correct.item() / data_iter.data_num
        print("Epoch {}, train loss: {:.5f}, train acc: {:.3f}".format(epoch, avg_loss, acc))
        dis_adversarial_train_loss.append(avg_loss)
        dis_adversarial_train_acc.append(acc)


def eval_discriminator(model, data_iter, criterion, args):
    """
    Evaluate discriminator, dropout is enabled
    """
    correct = 0
    total_loss = 0.
    with torch.no_grad():
        for data, target in data_iter:
            if args.cuda:
                data, target = data.cuda(), target.cuda()
            target = target.contiguous().view(-1)
            output = model(data)
            pred = output.data.max(1)[1]
            correct += pred.eq(target.data).cpu().sum()
            loss = criterion(output, target)
            total_loss += loss.item()
    avg_loss = total_loss / len(data_iter)
    acc = correct.item() / data_iter.data_num
    return avg_loss, acc


def adversarial_train(gen, dis, rollout, pg_loss, nll_loss, gen_optimizer, dis_optimizer, 
        dis_adversarial_train_loss, dis_adversarial_train_acc, args):
    """
    Adversarially train generator and discriminator
    """
    # train generator for g_steps
    print("#Train generator")
    for i in range(args.g_steps):
        print("##G-Step {}".format(i))
        train_generator_PG(gen, dis, rollout, pg_loss, gen_optimizer, args.gk_epochs, args)

    # train discriminator for d_steps
    print("#Train discriminator")
    for i in range(args.d_steps):
        print("##D-Step {}".format(i))
        train_discriminator(dis, gen, nll_loss, dis_optimizer, args.dk_epochs, 
            dis_adversarial_train_loss, dis_adversarial_train_acc, args)

    # update roll-out model
    rollout.update_params()

def translating_sample(sample_path , voca_path):

  print("================================================")
  print("생성된 데이터 확인.....")
  df = pd.read_csv(sample_path , names = ["data"] , sep = "\t" )
  Sentences = [sentence.split(" ") for sentence in df['data'].values]
  print("생성된 가사 수 : {} , 가사 시퀀스 수 : {} ".format(len(Sentences) , len(Sentences[0])))
  print(" ")

  print("저장된 단어사전 확인....")
  a = open(voca_path, 'rb')
  voca = pickle.load(a)
  print("단어 사전에 등록된 단어 수 : " , len(voca))
  print(" ")

  # 확인할 무작위 시드 / 사이즈
  random_seed = 100
  random_size = 10

  test_Sentences = Sentences[random_seed:random_seed + random_size]


  for sentence in test_Sentences:
    real_sentence = ""
    for s in sentence:

      try:
        s = int(s)
      except: pass

      if voca.get(s):
        if voca[s] == "UNK":
          break
        real_sentence += voca[s] + " "

    print(real_sentence)
    print("------------------------")

## SeqGAN 실행

### 세팅

In [17]:
# Arguemnts
parser = argparse.ArgumentParser(description='SeqGAN')
parser.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")
parser.add_argument('--hpc', action='store_true', default=False,
                    help='set to hpc mode')
parser.add_argument('--data_path', type=str, default='/content/data/', metavar='PATH',
                    help='data path to save files (default: /content/data/)')
parser.add_argument('--rounds', type=int, default=150, metavar='N',
                    help='rounds of adversarial training (default: 150)')
parser.add_argument('--g_pretrain_steps', type=int, default=10, metavar='N',
                    help='steps of pre-training of generators (default: 120)')
parser.add_argument('--d_pretrain_steps', type=int, default=50, metavar='N',
                    help='steps of pre-training of discriminators (default: 50)')
parser.add_argument('--g_steps', type=int, default=1, metavar='N',
                    help='steps of generator updates in one round of adverarial training (default: 1)')
parser.add_argument('--d_steps', type=int, default=3, metavar='N',
                    help='steps of discriminator updates in one round of adverarial training (default: 3)')
parser.add_argument('--gk_epochs', type=int, default=1, metavar='N',
                    help='epochs of generator updates in one step of generate update (default: 1)')
parser.add_argument('--dk_epochs', type=int, default=3, metavar='N',
                    help='epochs of discriminator updates in one step of discriminator update (default: 3)')
parser.add_argument('--update_rate', type=float, default=0.8, metavar='UR',
                    help='update rate of roll-out model (default: 0.8)')
parser.add_argument('--n_rollout', type=int, default=16, metavar='N',
                    help='number of roll-out (default: 16)')
parser.add_argument('--vocab_size', type=int, default=10365, metavar='N',
                    help='vocabulary size (default: 11292)')
parser.add_argument('--batch_size', type=int, default=64, metavar='N',
                    help='batch size (default: 64)')
parser.add_argument('--n_samples', type=int, default=6400, metavar='N',
                    help='number of samples gerenated per time (default: 6400)')
parser.add_argument('--gen_lr', type=float, default=1e-3, metavar='LR',
                    help='learning rate of generator optimizer (default: 1e-3)')
parser.add_argument('--dis_lr', type=float, default=1e-3, metavar='LR',
                    help='learning rate of discriminator optimizer (default: 1e-3)')
parser.add_argument('--no_cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')

_StoreAction(option_strings=['--seed'], dest='seed', nargs=None, const=None, default=1, type=<class 'int'>, choices=None, help='random seed (default: 1)', metavar='S')

In [18]:
# Parse arguments
args = parser.parse_args()
args.cuda = not args.no_cuda and torch.cuda.is_available()
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)
if not args.hpc:
    args.data_path = ''

# Positive_File / Negative_File
POSITIVE_FILE = args.data_path + POSITIVE_FILE
NEGATIVE_FILE = args.data_path + NEGATIVE_FILE

### 모델 로스 생성

In [19]:
# Set models, criteria, optimizers
generator = Generator(args.vocab_size, g_embed_dim, g_hidden_dim, args.cuda)
discriminator = Discriminator(d_num_class, args.vocab_size, d_embed_dim, d_filter_sizes, d_num_filters, d_dropout_prob)
# target_lstm = TargetLSTM(args.vocab_size, g_embed_dim, g_hidden_dim, args.cuda)

nll_loss = nn.NLLLoss()
pg_loss = PGLoss()

### cuda/optimizer 생성

In [20]:
if args.cuda:
    generator = generator.cuda()
    discriminator = discriminator.cuda()
    # target_lstm = target_lstm.cuda()
    nll_loss = nll_loss.cuda()
    pg_loss = pg_loss.cuda()
    cudnn.benchmark = True

gen_optimizer = optim.Adam(params=generator.parameters(), lr=args.gen_lr)
dis_optimizer = optim.SGD(params=discriminator.parameters(), lr=args.dis_lr)

### Pretrain Generator / Discriminator

In [21]:
# Container of experiment data
gen_pretrain_train_loss = []
gen_pretrain_eval_loss = []
dis_pretrain_train_loss = []
dis_pretrain_train_acc = []
dis_pretrain_eval_loss = []
dis_pretrain_eval_acc = []
gen_adversarial_eval_loss = []
dis_adversarial_train_loss = []
dis_adversarial_train_acc = []
dis_adversarial_eval_loss = []
dis_adversarial_eval_acc = []

In [ ]:
# # Generate toy data using target LSTM
# print('#####################################################')
# print('Generating data ...')
# print('#####################################################\n\n')
# generate_samples(target_lstm, args.batch_size, args.n_samples, POSITIVE_FILE)

#####################################################
Generating data ...
#####################################################




In [22]:
MODEL_PATH = "/content/generator.pt"
VOCA_PATH = "/content/data/idx2pos.pkl"
GENERATE_PATH = "/content/gene_data.txt"

# Pre-train generator using MLE
print('#####################################################')
print('Start pre-training generator with MLE...')
print('#####################################################\n')
gen_data_iter = GenDataIter(POSITIVE_FILE, args.batch_size)

#for i in range(args.g_pretrain_steps):
for i in range(1):      
    print("G-Step {}".format(i))
    train_generator_MLE(generator, gen_data_iter, nll_loss, 
        gen_optimizer, args.gk_epochs, 
        gen_pretrain_train_loss, args)
    generate_samples(generator, args.batch_size, args.n_samples, NEGATIVE_FILE)
    eval_iter = GenDataIter(NEGATIVE_FILE, args.batch_size)
    gen_loss = eval_generator(generator, eval_iter, nll_loss, args)
    gen_pretrain_eval_loss.append(gen_loss)
    print("eval loss: {:.5f}\n".format(gen_loss))
    translating_sample(GENERATE_PATH , VOCA_PATH)

print('#####################################################\n\n')

print("Pretrain Generator Model Save...!")
torch.save(generator , MODEL_PATH)


#####################################################
Start pre-training generator with MLE...
#####################################################

G-Step 0
Epoch 0, train loss: 3.81953
eval loss: 3.48328

생성된 데이터 확인.....
생성된 가사 수 : 6400 , 가사 시퀀스 수 : 20 
 
저장된 단어사전 확인....
단어 사전에 등록된 단어 수 :  10365
 
아플 삶 푸른 에게 머릿결 열정 순간 같아요 별 
------------------------

------------------------
살아요 내 려고 있었는데 저 는 넘친 가나 좋아 요 가는 슬픔 
------------------------
너무 바라만 보다가도 너 하지 와 봐 말 
------------------------
너무 화려한 너 의 진정 
------------------------
네 위 모두 오는 
------------------------
도무지 쓰면 소리 
------------------------
헛된 이 오늘 길 
------------------------
다시는 에 처음 긴 좋았었는지 모든 만 봐요 에 끝 건 
------------------------
그대 같이 나일 맘 을 멸망하네 이 땐 에 처럼 
------------------------
#####################################################


Pretrain Generator Model Save...!


In [42]:
VOCA_PATH = "/content/data/idx2pos.pkl"
GENERATE_PATH = "/content/gene_data.txt"

translating_sample(GENERATE_PATH , VOCA_PATH)

생성된 데이터 확인.....
생성된 가사 수 : 6400 , 가사 시퀀스 수 : 20 
 
저장된 단어사전 확인....
단어 사전에 등록된 단어 수 :  10365
 
아플 삶 푸른 고민 머릿결 열정 알 같아요 떠오르는 마음 
------------------------

------------------------
살아요 내 려고 있었는데 저 는 넘친 가나 좋아 요 가는 슬픔 
------------------------
놀라워라 바라만 약속 너 하지 와 봐 말 
------------------------
너무 화려한 너 에 진정 
------------------------
달라질 위 
------------------------
난 쓰면 소리 여기 터질듯 같은 의 딱 혹시 낼거야 는 그럼 에 하지도 할 할 지금 이 봐 <start> 
------------------------
헛된 이 오늘 길 
------------------------
다시는 끝나지 처음 미안해 좋았었는지 모든 만 봐요 에 쌓이던 건 
------------------------
그대 같이 나일 맘 을 멸망하네 이 건지 에 처럼 
------------------------


In [23]:
# Pre-train discriminator
print('#####################################################')
print('Start pre-training discriminator...')
print('#####################################################\n')
for i in range(5):
#for i in range(args.d_pretrain_steps):      
    print("D-Step {}".format(i))
    train_discriminator(discriminator, generator, nll_loss, 
        dis_optimizer, args.dk_epochs, 
        dis_adversarial_train_loss, dis_adversarial_train_acc, args)
    generate_samples(generator, args.batch_size, args.n_samples, NEGATIVE_FILE)
    eval_iter = DisDataIter(POSITIVE_FILE, NEGATIVE_FILE, args.batch_size)
    dis_loss, dis_acc = eval_discriminator(discriminator, eval_iter, nll_loss, args)
    dis_pretrain_eval_loss.append(dis_loss)
    dis_pretrain_eval_acc.append(dis_acc)
    print("eval loss: {:.5f}, eval acc: {:.3f}\n".format(dis_loss, dis_acc))
print('#####################################################\n\n')

#####################################################
Start pre-training discriminator...
#####################################################

D-Step 0
Epoch 0, train loss: 0.58275, train acc: 0.747
Epoch 1, train loss: 0.54524, train acc: 0.763
Epoch 2, train loss: 0.51284, train acc: 0.778
eval loss: 0.50015, eval acc: 0.788

D-Step 1
Epoch 0, train loss: 0.48372, train acc: 0.797
Epoch 1, train loss: 0.45440, train acc: 0.822
Epoch 2, train loss: 0.43637, train acc: 0.838
eval loss: 0.42974, eval acc: 0.844

D-Step 2
Epoch 0, train loss: 0.42190, train acc: 0.848
Epoch 1, train loss: 0.40771, train acc: 0.858
Epoch 2, train loss: 0.39750, train acc: 0.865
eval loss: 0.39436, eval acc: 0.867

D-Step 3
Epoch 0, train loss: 0.38625, train acc: 0.871
Epoch 1, train loss: 0.38282, train acc: 0.874
Epoch 2, train loss: 0.37228, train acc: 0.877
eval loss: 0.37089, eval acc: 0.879

D-Step 4
Epoch 0, train loss: 0.37488, train acc: 0.876
Epoch 1, train loss: 0.37077, train acc: 0.877
Epoc

In [44]:
generator = torch.load(MODEL_PATH)

generate_samples(generator, args.batch_size, args.n_samples, NEGATIVE_FILE)
eval_iter = GenDataIter(NEGATIVE_FILE, args.batch_size)
gen_loss = eval_generator(generator, eval_iter, nll_loss, args)
gen_pretrain_eval_loss.append(gen_loss)
print("eval loss: {:.5f}\n".format(gen_loss))

translating_sample(GENERATE_PATH , VOCA_PATH)

eval loss: 3.51196

생성된 데이터 확인.....
생성된 가사 수 : 6400 , 가사 시퀀스 수 : 20 
 
저장된 단어사전 확인....
단어 사전에 등록된 단어 수 :  10365
 
데리러 펼치고 내 뿐 한 매력 저 마음 소리쳐 랑 
------------------------
또 발그레 생각 들 에서 종일 평생 한 눈물 줘요 
------------------------
날 맞대 우우 이 영원한 자신 에서 
------------------------
이 타 세탁소 마음 하고 처럼 해주지 만난 때 
------------------------
농담 그대 이 묻어주던 나을 모든 바람 넌 한 기억 
------------------------
오직 우 의 해 나를 태웠는지 
------------------------
길가 너 그 걸 을 그냥 쉴 모습 처럼 
------------------------
하루 정말 가 그만 더 나눈 우주 이 움큼 난 않게 
------------------------
잊고 언젠가 그게 을 이내 잖아 내 들 가 내 나이 말 돌려주고 채워 세상 을 달라진 전 대도 
------------------------
그렇게 혼자 향 오늘 의 처음 이 먼저 무너지는 안되나 너 없을테니 
------------------------


In [ ]:
translating_sample(GENERATE_PATH , VOCA_PATH)

생성된 데이터 확인.....
생성된 가사 수 : 6400 , 가사 시퀀스 수 : 20 
 
저장된 단어사전 확인....
단어 사전에 등록된 단어 수 :  10365
 
울 어도 울 어도 네 가 돌아올 수 없다면 
------------------------
내 맘 에 네 온 날 느린 것 같아 외로움 
------------------------
누구 라도 모를거에요 창문 을 열어요 새 들 을 불러요 
------------------------

------------------------
행복해지는 동화 를 듣고 자라나지 아름답고 착한 사람 들 은 
------------------------
잎새 에 이는 바람 에도 난 괴롭다 했잖아 
------------------------
그대 왜 자꾸 웃어 줬나요 아무 상관없는 그대 삶 의 나라 면 
------------------------
당신 어깨 를 둘 수 있게 더 이상 아무 말 도 할 수가 없지만 
------------------------
아득히 멀기만 한 내 미래 를 비춰줄 끝없이 이끌어 줄 에 아름다운 마음 난 난 
------------------------
어쩌면 조금 우리 의 몸 을 지켜봐 주는 것 같아요 가슴 으로 느껴 보세요 
------------------------


### Adversarial Training

In [ ]:
# Adversarial training
print('#####################################################')
print('Start adversarial training...')
print('#####################################################\n')
rollout = Rollout(generator, args.update_rate)
#for i in range(1):    
for i in range(args.rounds):
    print("Round {}".format(i))
    adversarial_train(generator, discriminator, rollout, 
        pg_loss, nll_loss, gen_optimizer, dis_optimizer, 
        dis_adversarial_train_loss, dis_adversarial_train_acc, args)
    generate_samples(generator, args.batch_size, args.n_samples, NEGATIVE_FILE)
    gen_eval_iter = GenDataIter(NEGATIVE_FILE, args.batch_size)
    dis_eval_iter = DisDataIter(POSITIVE_FILE, NEGATIVE_FILE, args.batch_size)
    gen_loss = eval_generator(generator, gen_eval_iter, nll_loss, args)
    gen_adversarial_eval_loss.append(gen_loss)
    dis_loss, dis_acc = eval_discriminator(discriminator, dis_eval_iter, nll_loss, args)
    dis_adversarial_eval_loss.append(dis_loss)
    dis_adversarial_eval_acc.append(dis_acc)
    translating_sample(GENERATE_PATH , VOCA_PATH)
    print("gen eval loss: {:.5f}, dis eval loss: {:.5f}, dis eval acc: {:.3f}\n"
        .format(gen_loss, dis_loss, dis_acc))

# Save experiment data
with open(args.data_path + 'experiment.pkl', 'wb') as f:
    pickle.dump(
        (gen_pretrain_train_loss,
            gen_pretrain_eval_loss,
            dis_pretrain_train_loss,
            dis_pretrain_train_acc,
            dis_pretrain_eval_loss,
            dis_pretrain_eval_acc,
            gen_adversarial_eval_loss,
            dis_adversarial_train_loss,
            dis_adversarial_train_acc,
            dis_adversarial_eval_loss,
            dis_adversarial_eval_acc),
        f,
        protocol=pickle.HIGHEST_PROTOCOL
    )

#####################################################
Start adversarial training...
#####################################################

Round 0
#Train generator
##G-Step 0
#Train discriminator
##D-Step 0
Epoch 0, train loss: 0.26438, train acc: 0.908
Epoch 1, train loss: 0.25544, train acc: 0.912
Epoch 2, train loss: 0.24831, train acc: 0.917
##D-Step 1
Epoch 0, train loss: 0.24401, train acc: 0.919
Epoch 1, train loss: 0.24217, train acc: 0.921
Epoch 2, train loss: 0.24279, train acc: 0.921
##D-Step 2
Epoch 0, train loss: 0.24126, train acc: 0.922
Epoch 1, train loss: 0.24159, train acc: 0.922
Epoch 2, train loss: 0.23836, train acc: 0.924
생성된 데이터 확인.....
생성된 가사 수 : 6400 , 가사 시퀀스 수 : 20 
 
저장된 단어사전 확인....
단어 사전에 등록된 단어 수 :  10365
 
샤워 사는 로 일만 저 때 남자 만일 그냥 영원할 없나요 에 변함 고독 미친 가본 나를 하면서 우리 웃어 
------------------------
순식간 말 두 거리 이 퍼질 몰래 싶던 보내진 추억 마음 일 괜찮잖아 조그만 괴롭혔지 하는 그 흘려 침착하게 숨어가고 
------------------------
길 라라라라 기다리고 내 을 살아가지 오랜 가 를 버리겠어 충만 맡긴 때 이렇게 사랑 갤 있잖아 다 멀다하고 솔직하게 
----------

In [ ]:
# MODEL_PATH = "/content/generator_1.pt"

torch.save(generator, "/content/generator_2.pt")

In [ ]:
generator = torch.load("/content/generator_1.pt")

In [ ]:
translating_sample('/content/real_data.txt' , VOCA_PATH)

생성된 데이터 확인.....


AttributeError: ignored